# PostgreSQL データ確認

In [1]:
import os
from dotenv import load_dotenv
import psycopg2

# .env の読み込み
load_dotenv(override=True)

PG_HOST = os.getenv("PG_HOST")
PG_DB = os.getenv("PG_DB")
PG_USER = os.getenv("PG_USER")
PG_PASS = os.getenv("PG_PASS")

# テーブル一覧取得
def get_table_list(conn):
    with conn.cursor() as cur:
        cur.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
              AND table_type = 'BASE TABLE'
        """)
        return [row[0] for row in cur.fetchall()]

# カラム定義・件数
def describe_tables(conn, tables):
    infos = []
    for table in tables:
        with conn.cursor() as cur:
            cur.execute("""
                SELECT column_name, data_type
                FROM information_schema.columns
                WHERE table_name = %s
            """, (table,))
            columns = cur.fetchall()
            cur.execute(f'SELECT COUNT(*) FROM "{table}"')
            row_count = cur.fetchone()[0]
        infos.append({'table': table, 'columns': columns, 'row_count': row_count})
    return infos

# 結果表示
def show_table_summary(infos):
    for info in infos:
        print(f"--- {info['table']} ---")
        print("Columns:")
        for col, dtype in info['columns']:
            print(f"  {col} : {dtype}")
        print(f"Row count: {info['row_count']}\n")

# 接続＆実行
conn = psycopg2.connect(host=PG_HOST, dbname=PG_DB, user=PG_USER, password=PG_PASS)
tables = get_table_list(conn)
infos = describe_tables(conn, tables)
show_table_summary(infos)
conn.close()


--- inventory ---
Columns:
  inventory_id : integer
  product_id : integer
  stock : integer
  restock_date : text
Row count: 100

--- shipping_status ---
Columns:
  order_id : integer
  user_id : integer
  shipping_status : text
  shipping_date : text
  delivery_date : text
Row count: 70

--- order_details ---
Columns:
  subtotal : integer
  order_id : integer
  product_id : integer
  order_detail_id : integer
  unit_price : integer
  quantity : integer
  product_name : text
Row count: 135

--- products ---
Columns:
  product_id : integer
  category_id : integer
  price : integer
  product_name : text
  category_name : text
  created_at : text
Row count: 100

--- users ---
Columns:
  user_id : integer
  user_name : text
  email : text
  registered_at : text
Row count: 10

--- orders ---
Columns:
  order_id : integer
  user_id : integer
  total_amount : integer
  order_date : text
Row count: 70

--- categories ---
Columns:
  category_id : integer
  category_name : text
Row count: 5



# Cosmos DB データ確認

In [2]:
import os
from dotenv import load_dotenv
from azure.cosmos import CosmosClient
import pandas as pd

# .env 読み込み
load_dotenv(override=True)
COSMOS_ENDPOINT = os.getenv("COSMOS_ENDPOINT")
COSMOS_KEY = os.getenv("COSMOS_KEY")
COSMOS_DB = os.getenv("COSMOS_DB")
COSMOS_CONTAINER = os.getenv("COSMOS_CONTAINER")

# クライアント接続
client = CosmosClient(COSMOS_ENDPOINT, COSMOS_KEY)
db = client.get_database_client(COSMOS_DB)
container = db.get_container_client(COSMOS_CONTAINER)

# 件数確認
count = container.query_items(
    query="SELECT VALUE COUNT(1) FROM c",
    enable_cross_partition_query=True
)
total_docs = list(count)[0]
print(f"Document count: {total_docs}")

# 先頭10件サンプル取得
docs = list(container.query_items(
    query="SELECT * FROM c OFFSET 0 LIMIT 10",
    enable_cross_partition_query=True
))

# pandasで表示
if docs:
    df = pd.DataFrame(docs)
    display(df)
else:
    print("No documents found.")


Document count: 1526


,id,created_at,user,text,favorite_count,retweet_count,reply_count,quote_count,product_name,_rid,_self,_etag,_attachments,_ts
0,1,2024-01-01,{'screen_name': 'user09'},USB-C Charger 3がセール中！ #買ってよかった,7,0,0,0,USB-C Charger 3,-a5nAJX5AMEBAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00cde9-0000-0700-0000-688239050000""",attachments/,1753364741
1,2,2024-01-01,{'screen_name': 'user08'},Surface スリム ペン 4がセール中！ #買ってよかった,20,7,1,2,Surface スリム ペン 4,-a5nAJX5AMECAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00cee9-0000-0700-0000-688239050000""",attachments/,1753364741
2,3,2024-01-01,{'screen_name': 'user02'},今日のおすすめはWindows Server 4！,16,2,3,1,Windows Server 4,-a5nAJX5AMEDAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00d8e9-0000-0700-0000-688239050000""",attachments/,1753364741
3,4,2024-01-01,{'screen_name': 'user06'},やっぱりModern Webcam 5最高。#欲しいものリスト,18,1,5,0,Modern Webcam 5,-a5nAJX5AMEEAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00dde9-0000-0700-0000-688239060000""",attachments/,1753364742
4,5,2024-01-02,{'screen_name': 'user10'},Surface Pro 2がセール中！ #買ってよかった,20,9,2,1,Surface Pro 2,-a5nAJX5AMEFAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00e2e9-0000-0700-0000-688239060000""",attachments/,1753364742
5,6,2024-01-02,{'screen_name': 'user04'},やっぱりSurface ペン 2最高。#欲しいものリスト,4,10,1,1,Surface ペン 2,-a5nAJX5AMEGAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00e9e9-0000-0700-0000-688239060000""",attachments/,1753364742
6,7,2024-01-02,{'screen_name': 'user02'},今日のおすすめはWindows Server 4！,13,4,3,0,Windows Server 4,-a5nAJX5AMEHAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00efe9-0000-0700-0000-688239060000""",attachments/,1753364742
7,8,2024-01-02,{'screen_name': 'user05'},話題のSurface ペン 3を買いました！ #Microsoft,7,0,1,0,Surface ペン 3,-a5nAJX5AMEIAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00f4e9-0000-0700-0000-688239060000""",attachments/,1753364742
8,9,2024-01-03,{'screen_name': 'user05'},やっぱりXbox コントローラー 5最高。#欲しいものリスト,2,1,4,2,Xbox コントローラー 5,-a5nAJX5AMEJAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f00fce9-0000-0700-0000-688239070000""",attachments/,1753364743
9,10,2024-01-03,{'screen_name': 'user06'},やっぱりXbox コントローラー 1最高。#欲しいものリスト,10,7,4,2,Xbox コントローラー 1,-a5nAJX5AMEKAAAAAAAAAA==,dbs/-a5nAA==/colls/-a5nAJX5AME=/docs/-a5nAJX5A...,"""0f0000ea-0000-0700-0000-688239070000""",attachments/,1753364743
